# Name - Sachid Deshmukh
### Date: 06/19/2019
### Data-612 Assignment-2
##### Reference - http://www.salemmarafi.com/code/collaborative-filtering-with-python/

#### In this assignment we will use Last FM data set
#### The data set contains information about users, gender, age, and which artists they have listened to on Last.FM.

In [32]:
# import required packages
import pandas as pd
from scipy.spatial.distance import cosine

In [33]:
#Load Data in pandas dataframe. Restrict data for 50 users and 50 items
filepath = r'C:\MSDS\MSDSQ3Data612\Assignment-2\data.csv'
data = pd.read_csv(filepath)
data = data.iloc[:50,:50]
data.shape
# We have data for 50 users and 50 songs

(50, 50)

#### Item to Item Filtering
#### Drop user column since it is irrelevent for item to item filtering

In [34]:
data_item = data.drop('user', 1)
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=data_item.columns,columns=data_item.columns)

In [35]:
# Lets fill in those empty spaces with cosine similarities
# Loop through the columns
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_item.iloc[:,i],data_item.iloc[:,j])


C:\ProgramData\Anaconda3\lib\site-packages\scipy\spatial\distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [36]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index
 

#### Item to item collaborative filtering is completed. Let's preview 50 items along with 10 most similar items

In [37]:
data_neighbours.head(50)

,1,2,3,4,5,6,7,8,9,10
a perfect circle,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys
abba,abba,atb,amy macdonald,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica,anti-flag,amy winehouse
ac/dc,ac/dc,audioslave,beatsteaks,bloodhound gang,amy winehouse,atb,as i lay dying,arctic monkeys,arch enemy,arcade fire
adam green,adam green,belle and sebastian,bring me the horizon,amy winehouse,atb,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica
aerosmith,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys
afi,afi,anti-flag,bad religion,blink-182,bloodhound gang,billy talent,bring me the horizon,amy macdonald,arctic monkeys,arch enemy
air,air,bjork,atb,arcade fire,alanis morissette,beck,beastie boys,amy winehouse,beatsteaks,bloc party
alanis morissette,alanis morissette,bjork,bad religion,air,atb,amy winehouse,as i lay dying,arctic monkeys,arch enemy,arcade fire
alexisonfire,alexisonfire,breaking benjamin,bring me the horizon,bob marley,as i lay dying,billy talent,atb,air,alanis morissette,afi
alicia keys,alicia keys,bring me the horizon,amy macdonald,atb,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica,anti-flag


### The process for creating a User Based recommendation system is as follows:

#### Have an Item Based similarity matrix at your disposal
#### Check which items the user has consumed
#### For each item the user has consumed, get the top X neighbours
#### Get the consumption record of the user for each neighbour.
#### Calculate a similarity score using some formula
#### Recommend the items with the highest score

In [38]:
# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [39]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data.index,columns=data.columns)
data_sims.iloc[:,:1] = data.iloc[:,:1]

In [40]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if data.iloc[i][j] == 1:
            data_sims.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            user_purchases = data_item.loc[user,product_top_names]
 
            data_sims.iloc[i][j] = getScore(user_purchases,product_top_sims)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
# Get the top songs
data_recommend = pd.DataFrame(index=data_sims.index, columns=['user','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]

# Instead of top song scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

#### User to user collaborative filtering is completed. lets view top 6 recommended items for each users based on their past listening history

In [42]:
data_recommend.head(50)

,user,1,2,3,4,5,6
0,1,atreyu,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica
1,33,atreyu,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica
2,42,bob marley,arch enemy,apocalyptica,bring me the horizon,all that remains,alexisonfire
3,51,bob dylan,blink-182,ac/dc,atreyu,breaking benjamin,bloc party
4,62,bob dylan,arctic monkeys,ac/dc,afi,anti-flag,atreyu
5,75,abba,bob dylan,atreyu,bjork,bring me the horizon,bloodhound gang
6,130,atreyu,as i lay dying,arctic monkeys,arch enemy,arcade fire,apocalyptica
7,141,avril lavigne,bring me the horizon,arch enemy,apocalyptica,ac/dc,alexisonfire
8,144,bright eyes,arcade fire,bloc party,abba,amy macdonald,as i lay dying
9,150,bob dylan,beatsteaks,bloc party,black eyed peas,blink-182,billy talent
